### Imports

In [16]:
import pandas
import numpy
import wfdb
import ast

### Paths

In [17]:
path = '/home/rishikant/Documents/Datasets/PTB-XL/'

### Loading and converting raw annotations

In [18]:
Y = pandas.read_csv(path + 'ptbxl_database.csv', index_col = 'ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

### Loading raw signal data

In [19]:
X = [wfdb.rdsamp(path + f, channels=[0]) for f in Y.filename_hr]
X = numpy.array([signal.flatten() for signal, meta in X]).T

### Loading diagnostic statements

In [ ]:
agg_df = pandas.read_csv(path + 'scp_statements.csv', index_col = 0)
agg_df = agg_df[agg_df.diagnostic == 1]

### Aggregating diagnostic information as superclass

In [ ]:
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

### Saving the data
- Waveforms are saved as numpy array
- Annotations are saved as pandas dataframe

In [ ]:
# Waveforms
numpy.savetxt('waveforms.csv', X, delimiter=',')

# Annotations
Y['diagnostic_superclass'].to_csv('annotations.csv')